# Regrid time-averaged LFRic data

In [1]:
import warnings

warnings.filterwarnings("ignore")

from functools import partial
from pathlib import Path

import iris
import numpy as np
import paths
from aeolus.calc import last_n_day_mean, time_mean, zonal_mean
from aeolus.const import add_planet_conf_to_cubes, init_const
from aeolus.io import create_dummy_cube, load_data, save_cubelist
from aeolus.lfric import (
    add_um_height_coord,
    fix_time_coord,
    load_lfric_raw,
    simple_regrid_lfric,
    ugrid_spatial,
)
from aeolus.model import lfric
from common import SIMULATIONS, SPINUP_DAYS

import os

os.environ["PROJ_IGNORE_CELESTIAL_BODY"] = "YES"

In [2]:
sim_label = "c192s10r"
# const = init_const(planet, directory=paths.const)

In [3]:
grid_cell_area = load_lfric_raw(
    paths.data_raw
    / SIMULATIONS[sim_label].work_name
    / "1"
    / f"run*"
    / "lfric_initial.nc",
).extract_cube("grid_cell_area")
grid_cell_area.attributes.pop("invalid_standard_name");
# cell_width = grid_cell_area**0.5
# cell_width.convert_units("km")
# cell_width.rename("grid_cell_width")
# cell_width.attributes.pop("invalid_standard_name");

In [4]:
# time_prof = "inst_diag"
time_prof = "averages"

In [5]:
add_levs = partial(
    add_um_height_coord,
    path_to_levels_file=paths.vert / f"vertlevs_{SIMULATIONS[sim_label].vert_lev}",
)


def combi_callback(cube, field, filename):
    [
        fix_time_coord(cube, field, filename),
        add_levs(cube, field, filename),
    ]

In [6]:
fnames = sorted(
    paths.data_raw.glob(
        str(
            Path(SIMULATIONS[sim_label].work_name)
            / "*"
            / "run_lfric_atm_*"
            / f"lfric_{time_prof}.nc"
        )
    ),
    key=lambda x: int(x.parent.parent.name),
)

fnames = [
    i
    for i in fnames
    if int(i.parent.parent.name) * SIMULATIONS[sim_label].days_per_job > SPINUP_DAYS
]

In [7]:
dset_raw = load_lfric_raw(
    fnames,
    callback=combi_callback,
    drop_coord=["forecast_reference_time"],
)
for cube in dset_raw:
    if cube.units == "ms-1":
        cube.units = "m s-1"

for i, cube in enumerate(dset_raw):
    print(f"---|---|-{'-'*30}-|-{'-'*60}")
    print(f"{i:<2d} | {cube.ndim} | {cube.var_name:>30} | {cube.name():>60}")

---|---|--------------------------------|-------------------------------------------------------------
0  | 3 |                          theta |                                    air_potential_temperature
---|---|--------------------------------|-------------------------------------------------------------
1  | 2 |      tot_col_uv_kinetic_energy |                 atmosphere_horizontal_kinetic_energy_content
---|---|--------------------------------|-------------------------------------------------------------
2  | 2 |                   tot_col_m_ci |                         atmosphere_mass_content_of_cloud_ice
---|---|--------------------------------|-------------------------------------------------------------
3  | 2 |                   tot_col_m_cl |                atmosphere_mass_content_of_cloud_liquid_water
---|---|--------------------------------|-------------------------------------------------------------
4  | 2 |                    tot_col_m_v |                       atmospher

In [8]:
cubes_to_regrid = time_mean(dset_raw, model=lfric)
cubes_to_regrid.append(grid_cell_area)

In [9]:
n_res = 512
tgt_cube = create_dummy_cube(n_res=n_res, pm180=True)

In [10]:
%%time
dset_regr = simple_regrid_lfric(
    cubes_to_regrid,
    tgt_cube=tgt_cube,
    ref_cube_constr=lfric.thta if time_prof == "averages" else lfric.caf,
    interp_vertically=(time_prof == "averages"),
)

for i, cube in enumerate(dset_regr):
    print(f"---|---|-{'-'*30}-|-{'-'*60}")
    print(f"{i:<2d} | {cube.ndim} | {cube.var_name:>30} | {cube.name():>60}")

---|---|--------------------------------|-------------------------------------------------------------
0  | 3 |                          theta |                                    air_potential_temperature
---|---|--------------------------------|-------------------------------------------------------------
1  | 2 |      tot_col_uv_kinetic_energy |                 atmosphere_horizontal_kinetic_energy_content
---|---|--------------------------------|-------------------------------------------------------------
2  | 2 |                   tot_col_m_ci |                         atmosphere_mass_content_of_cloud_ice
---|---|--------------------------------|-------------------------------------------------------------
3  | 2 |                   tot_col_m_cl |                atmosphere_mass_content_of_cloud_liquid_water
---|---|--------------------------------|-------------------------------------------------------------
4  | 2 |                    tot_col_m_v |                       atmospher

In [11]:
const = init_const(SIMULATIONS[sim_label].planet, directory=paths.const)
add_planet_conf_to_cubes(dset_regr, const=const)

In [12]:
# Write the data to a netCDF file
outdir = paths.data_proc / SIMULATIONS[sim_label].work_name
outdir.mkdir(parents=True, exist_ok=True)
gl_attrs = {
    "name": SIMULATIONS[sim_label].work_name,
    "planet": SIMULATIONS[sim_label].planet,
    "processed": "True",
}
chunk_label = f"_{int(fnames[0].parent.parent.name):03d}"
chunk_label += f"-{int(fnames[-1].parent.parent.name):03d}"
fname_out = (
    outdir
    / f"{SIMULATIONS[sim_label].work_name}_{time_prof}_{chunk_label}_time_mean_and_regr_{n_res}.nc".lower()
)
save_cubelist(dset_regr, fname_out, **gl_attrs)
print(f"Saved to: {str(fname_out)}")

Saved to: /home/ds591/exoplanets/stretched_mesh_proj/data/proc/thai_hab1_c192s10_l63_rc/thai_hab1_c192s10_l63_rc_averages__101-200_time_mean_and_regr_512.nc


## Tidally-locked coordinates

In [9]:
from aeolus.coord import get_xy_coords
from iris.analysis.cartography import rotate_pole, rotate_winds
from iris.coords import AuxCoord
from iris.cube import Cube
from iris.util import promote_aux_coord_to_dim_coord


def regrid_ugrid_to_rotated_pole_coordinates(
    cube,
    tgt_cube,
    pole_lon,
    pole_lat,
    model=lfric,
):
    """
    Regrid a UGRID cube to rotated-pole coordinates.

    Parameters
    ----------
    cube: iris.cube.Cube
        Input cube on a UGRID mesh.
    tgt_cube: iris.cube.Cube
        Cube on a lat-lon grid with the target resolution.
    pole_lon: float
        New North Pole longitude.
    pole_lat: float
        New North Pole latitude.
    model: aeolus.model.Model, optional
        Model class with relevant coordinate and variable names.
    """
    _cs = tgt_cube.coord_system()
    xcoord, ycoord = get_xy_coords(cube, model=model)
    r_lons, r_lats = rotate_pole(xcoord.points, ycoord.points, pole_lon, pole_lat)

    r_lon_coord = AuxCoord(
        r_lons,
        units=xcoord.units,
        standard_name=xcoord.standard_name,
        var_name=xcoord.var_name,
        coord_system=_cs,
    )
    r_lat_coord = AuxCoord(
        r_lats,
        units=ycoord.units,
        standard_name=ycoord.standard_name,
        var_name=ycoord.var_name,
        coord_system=_cs,
    )
    non_xy_dim_coords = [
        (c, cube.coord_dims(c)) for c in cube.dim_coords if c not in [xcoord, ycoord]
    ]
    aux_coords = [
        (c, cube.coord_dims(c))
        for c in cube.aux_coords
        if isinstance(c, AuxCoord) and c not in [xcoord, ycoord]
    ]
    # assume latitude and longitude are the rightmost dimensions
    lat_dim = len(non_xy_dim_coords)
    if lat_dim != cube.coord_dims(ycoord)[0]:
        raise BadCoordinateError(
            "Ensure latitude and longitude are the rightmost dimensions."
        )
    cube_rot = Cube(
        data=cube.core_data(),
        aux_coords_and_dims=[
            *non_xy_dim_coords,
            *aux_coords,
            (r_lat_coord, lat_dim),
            (r_lon_coord, lat_dim),
        ],
        units=cube.units,
    )
    cube_rot.metadata = cube.metadata
    for coord, _ in non_xy_dim_coords:
        promote_aux_coord_to_dim_coord(cube_rot, coord)

    out = cube_rot.regrid(tgt_cube, iris.analysis.UnstructuredNearest())
    return out

In [10]:
# from aeolus.coord import get_xy_coords
# from iris.analysis.cartography import rotate_pole
# from iris.coords import AuxCoord
# from iris.cube import Cube
# from iris.util import promote_aux_coord_to_dim_coord


# def regrid_ugrid_to_rotated_pole_coordinates2(
#     cube,
#     tgt_cube,
#     pole_lon,
#     pole_lat,
#     model=lfric,
#     method="conservative",
# ):
#     """
#     Regrid a UGRID cube to rotated-pole coordinates.

#     Note: non-dimensional coordinates are lost.

#     Parameters
#     ----------
#     cube: iris.cube.Cube
#         Input cube on a UGRID mesh.
#     tgt_cube: iris.cube.Cube
#         Cube on a lat-lon grid with the target resolution.
#     pole_lon: float
#         New North Pole longitude.
#     pole_lat: float
#         New North Pole latitude.
#     model: aeolus.model.Model, optional
#         Model class with relevant coordinate and variable names.
#     """
#     from esmf_regrid.experimental.unstructured_scheme import MeshToGridESMFRegridder

#     xcoord, ycoord = get_xy_coords(cube, model=model)
#     r_lons, r_lats = rotate_pole(xcoord.points, ycoord.points, pole_lon, pole_lat)

#     r_lon_coord = xcoord.copy(points=r_lons)
#     r_lat_coord = ycoord.copy(points=r_lats)

#     regridder = MeshToGridESMFRegridder(cube, tgt_cube, method=method)
#     out = regridder(cube_flat)
#     return out

In [11]:
from aeolus.lfric import replace_level_coord_with_height
from iris.cube import CubeList
from typing import Optional
from aeolus.model.base import Model


def regrid_lfric_to_rotated_lat_lon(
    cube_list: CubeList,
    tgt_cube: Cube,
    pole_lon: float,
    pole_lat: float,
    ref_cube_constr: Optional[str] = "air_potential_temperature",
    model: Optional[Model] = lfric
) -> CubeList:
    """Nearest-method regridding of LFRic data to a common Rotated Pole height/lat/lon grid."""
    # Horizontal regridding
    result_h = CubeList()
    ref_cube = cube_list.extract_cube(ref_cube_constr)
    for cube in cube_list:
        # if cube.name() in [model.u, model.v]:
        #     print(f"Skipping {cube.name()}")
        # else:
        cube_reg = regrid_ugrid_to_rotated_pole_coordinates(cube, tgt_cube, pole_lon, pole_lat, model=model)
        result_h.append(cube_reg)

    # Vertical interpolation
    result_v = CubeList()
    tgt_points = (model.z, ref_cube.coord(model.z).points)
    for cube in result_h:
        if len(
            [i.name() for i in cube.dim_coords if i.name().endswith("_levels")]
        ):
            result_v.append(
                replace_level_coord_with_height(cube).interpolate(
                    [tgt_points], iris.analysis.Linear()
                )
            )
        else:
            result_v.append(cube)
    return result_v

In [12]:
%%time
dset_tl = regrid_lfric_to_rotated_lat_lon(
    cubes_to_regrid, tgt_cube, 0, 0, ref_cube_constr=lfric.thta
)
dset_tl = CubeList([zonal_mean(cube) for cube in dset_tl])

KeyboardInterrupt: 

In [13]:
const = init_const(SIMULATIONS[sim_label]["planet"], directory=paths.const)
add_planet_conf_to_cubes(dset_tl, const=const)

In [14]:
# Write the data to a netCDF file
outdir = paths.data_proc / SIMULATIONS[sim_label]["long_name"]
outdir.mkdir(parents=True, exist_ok=True)
gl_attrs = {
    "name": SIMULATIONS[sim_label]["long_name"],
    "planet": SIMULATIONS[sim_label]["planet"],
    "processed": "True",
}
chunk_label = f"_{int(fnames[0].parent.parent.name):03d}"
chunk_label += f"-{int(fnames[-1].parent.parent.name):03d}"
fname_out = (
    outdir
    / f"{SIMULATIONS[sim_label].work_name}_{time_prof}_{chunk_label}_tl_coord__time_and_zonal_mean.nc".lower()
)
save_cubelist(dset_tl, fname_out, **gl_attrs)